# Video Processing for BT Lab
Video files to png
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [2]:
#import libraries
import os
import re
import cv2
import json
import pandas as pd

In [3]:
video_path = "D:/Christian/DI_centre_structured"
repo_dir = os.getcwd()
json_dir = repo_dir + "/records/JSON"

In [19]:
def run_video_to_frame(video_path:str , save_folder:str , step: int):
    video = cv2.VideoCapture(video_path)
    assert video.isOpened()

    frame_count = 0
    saved_count = 0
    still_has_frames = True

    try:
        while still_has_frames:
            still_has_frames, frame = video.read()
            frame_name = "frame_%d.png" % saved_count
            frame_folder = save_folder + "/" + frame_name
            cv2.imwrite(frame_folder, frame)
            frame_count += step
            video.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
            saved_count += 1
    except Exception as e:
        print(f"{type(e).__name__}: {e}, frame_count: {frame_count}, saved_count: {saved_count}")

    video.set(cv2.CAP_PROP_POS_FRAMES, 0)
    video.release()

video_file = "D:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Hold Breath/Arun2.mp4"
save_folder = "D:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Hold Breath/frames"
run_video_to_frame(video_path, save_folder, 2)

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:783: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'
, frame_count: 0, saved_count: 0


In [17]:
video = cv2.VideoCapture(video_file)
video.release()

In [13]:
def get_frames_path(local_path: str):
    new_path = local_path.replace("\\", "/")
    new_path = new_path.split("/")[:-1]
    new_path = "/".join(new_path) + "/frames"
    return new_path

In [6]:
def convert_video_to_frame(all_patients:dict, target_fps):
    for patient_id, patient_info in all_patients.items():
        try:
            video_path = patient_info["local path"]
            frames_folder = get_frames_path(video_path)
            new_fps = int(patient_info["old fps"]/target_fps)
            print(frames_folder)
            if new_fps == 0:
                raise ValueError()

            run_video_to_frame(video_path, frames_folder, new_fps)
        except Exception as e:
            print(f"{type(e)} - choose lower bound fps for {video_path}")
        break;

In [7]:
def load_json(json_dir:str, filename:str) -> dict:
    full_path = json_dir + "/" + filename

    with open(full_path, "r") as json_data:
        data = json.load(json_data)

    return(data)

## Video to frames (stage 3.1)

Based on the desired fps, turn videos into frames

In [8]:
""" local vals"""

target_rgb_fps_lower = 10
target_rgb_fps_upper = 20
target_thermal_fps_lower = 5
target_thermal_fps_upper = 210

In [9]:
""" load JSON files """

metadata_rgb = load_json(json_dir, "rgb_complete.json")
metadata_thermal = load_json(json_dir, "thermal_complete.json")

In [14]:
""" convert video to png (rgb) """
convert_video_to_frame(metadata_rgb, target_rgb_fps_lower)

D:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Hold Breath/frames
error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:783: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'
, frame_count: 10, saved_count: 5
